<a href="https://colab.research.google.com/github/MichaelSeitz98/Seminararbeit/blob/main/Schwellenwertanalyse_Inferenzanwendung_Bild_mit_Einteilung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Tensorflow OD API installieren

In [2]:
# Tensorflow installieren

!pip install tensorflow=="2.6.0"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 564.4 MB 2.2 kB/s 
     |████████████████████████████████| 14.8 MB 5.1 MB/s 
     |████████████████████████████████| 132 kB 71.1 MB/s 
  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30694 sha256=3d340dba9fcb9f6a33e588ae3079980b394628654dcc12797f7eb0000386f0b9
  Stored in directory: /root/.cache/pip/wheels/98/91/04/971b4c587cf47ae952b108949b46926f426c02832d120a082a
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4849 sha256=5f025ca676996e1d4b06015937d3d6922a733340307aa53bfcca0d1030e65a75
  Stored in directory: /root/.cache/pip/wheels/3f/e3/ec/8a8336ff196023622fbcb36de0c5a5c218cbb24111d1d4c7f2
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68721 sha256=5ff2d21b6cbc57f92b17c152dfd1f0d2e352d4093dc5a1b717354493c2611afb
  Stored in directory: /root/.cache/pip/wheels/62/7

In [3]:
# Tensorflow Models klonen, wenn es noch nicht vorhanden ist

import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3529, done.
remote: Counting objects: 100% (3529/3529), done.
remote: Compressing objects: 100% (2941/2941), done.
remote: Total 3529 (delta 928), reused 1437 (delta 534), pack-reused 0
Receiving objects: 100% (3529/3529), 47.00 MiB | 27.90 MiB/s, done.
Resolving deltas: 100% (928/928), done.


In [4]:
# Installieren der Object Detection API 

%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1696454 sha256=02ddfa5d0c4615e41a6b617161b73642fce796ae62bd07727c9bcb63d0545604
  Stored in directory: /tmp/pip-ephem-wheel-cache-2hi6jmb2/wheels/fa/a4/d2/e9a5057e414fd46c8e543d2706cd836d64e1fcd9eccceb2329
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78545 sha256=e2666c70ff5cd1969170bd962de0242d82730dc1adf0f594154fb421b3f6abf3
  Stored in directory: /root/.cache/pip/wheels/a4/61/fd/c57e374e580aa78a45ed78d5859b3a44436af17e22ca53284f
  Created wheel for avro-python3: filename=avro_python3-1.10.2-py3-none-any.whl size=44010 sha256=3db8f61ed8b5b05dbbd1ee76cee192297c08b95ec18d5596318618432f49c17f
  Stored in directory: /root/.cache/pip/wheels/d6/e5/b1/6b151d9b535ee50aaa6ab27d145a0104b6df02e5636f0376da
  Created wheel for docopt: filename=docopt-

  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.


In [ ]:
# Installation testen 

!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

2022-11-14 13:14:32.228841: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-14 13:14:32.442500: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-14 13:14:33.776006: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-11-14 13:14:33.776253: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinf

# Modell & Labelmap laden

Modell und LabelMap von meinem GitHub-Account laden, dass auch auch ohne Google Drive Zugriff ausführbar

In [ ]:
# LabelMap von meinem GitHub-Account laden

%cd /content/
!wget https://raw.githubusercontent.com/MichaelSeitz98/Seminararbeit/main/labelmap.pbtxt

In [ ]:
labelmap_path = '/content/labelmap.pbtxt'

saved_model_dir = '/content/drive/MyDrive/training2_effdetd1_ds2_8_20000_100/inference_graph/saved_model'
speicherPfadEvalBilder = '/content/drive/MyDrive/training2_effdetd1_ds2_8_20000_100/angewendeteBilder_train'
schwellenwertanaye_speicherPfadEvalBilder = '/content/drive/MyDrive/training2_effdetd1_ds2_8_20000_100/schwellenwertanalyse_val/'

OUTPUT_PATH_DANGER = f'{speicherPfadEvalBilder}/Danger'
OUTPUT_PATH_NO_DANGER = f'{speicherPfadEvalBilder}/No_Danger'

print (OUTPUT_PATH_DANGER)

if (not os.path.isdir(schwellenwertanaye_speicherPfadEvalBilder)): 
    %mkdir {speicherPfadEvalBilder}
    print(f'Ordner erzeugt: {schwellenwertanaye_speicherPfadEvalBilder}')
else:
    print(f'Ordner: {schwellenwertanaye_speicherPfadEvalBilder} existiert schon')

%mkdir {OUTPUT_PATH_DANGER}
%mkdir {OUTPUT_PATH_NO_DANGER}

input_bilder_pfad_train = '/content/drive/MyDrive/DS2_POOLG_80_20_20/train'
input_bilder_pfad_val ='/content/drive/MyDrive/DS2_POOLG_80_20_20/validation'
input_bilder_pfad_test ='/content/drive/MyDrive/DS2_POOLG_80_20_20/test'


In [ ]:
# Alle benötigte packages für die Inferenz importiert

import io
import os
import scipy.misc
import numpy as np
import six
import time
import glob
from IPython.display import display

from six import BytesIO

import matplotlib
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

%matplotlib inline

In [ ]:
# Methode um aus Image ein Numpy_array zu machen 

def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path (this can be local or on colossus)

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
# Label map Category Index erzeugen
category_index = label_map_util.create_category_index_from_labelmap(labelmap_path, use_display_name=True)

print(f'Catergory Index erzeugt aus LabelMap: {labelmap_path}')

In [ ]:
# TODO: noch saved_model_dir zuerst über wget aus meinem GitHub-Account laden
# Das Modell wird hier reingeladen. Kann bis zu 2 Minuten dauern. 

tf.keras.backend.clear_session()
model = tf.saved_model.load(saved_model_dir)

print(f'Trainiertes Model erfolgreich geladen von: {saved_model_dir}')


In [ ]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  input_tensor = tf.convert_to_tensor(image)
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [ ]:
# Ab hier geht dann die Erkennung los
# Jedes Bild aus Test-Eval-Ordner wird untersucht
# und in den SpeicherPfadEvalBilder abgespeichert. 

bilderCounter = 0
danger = False
CONFIDENT_SCORE_THRESHOLD_CHILD = 0.50




for image_path in glob.glob(f'{input_bilder_pfad_test}/*.jpg'):
  danger = False
  print(f'aktueller image_path:{image_path}')
  start_time_pro_bild = time.time()
  bilderCounter+=1
  image_name = os.path.basename(image_path).replace(".jpg", "")
  image_np = load_image_into_numpy_array(image_path)
  output_dict = run_inference_for_single_image(model, image_np)

  # Postprocessing

  classes_child = output_dict['detection_classes']
  scores_child = output_dict['detection_scores']
  boxes_child = output_dict["detection_boxes"]

  for i in range(0, len(classes_child)):
      if classes_child[i] == 1 and scores_child[i] >= CONFIDENT_SCORE_THRESHOLD_CHILD:
              danger = True
              print ("Dangerous! Child detected!")

  outImg = vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      min_score_thresh=CONFIDENT_SCORE_THRESHOLD_CHILD,
      max_boxes_to_draw=15,
      line_thickness=3)
  
  im = Image.fromarray(outImg)
  #display(Image.fromarray(image_np))
    
  if (danger):
        im.save(f'{OUTPUT_PATH_DANGER}/{image_name}_boxed_danger.jpg')
        print ('Hier gespeichert: ' + OUTPUT_PATH_DANGER + "/" + image_name + "_boxed.jpg")
  else:
        print('Keine Gefahr. Kein Kind entdeckt')
        im.save(f'{OUTPUT_PATH_NO_DANGER}/{image_name}_boxed_no_danger.jpg')
        print(image_name + " gespeichert: No Danger.\n")

print(f'Fertig! Modell auf {bilderCounter} Bilder angewendet und unter {speicherPfadEvalBilder} gespeichert.')